In [ ]:
'''
Copyright (c) Microsoft Corporation. All rights reserved.
Licensed under the MIT License.
'''
### Run the following command to install required packages:
#! pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
import os
import wave
import re
from collections import Counter
import cv2
import pylab
import matplotlib
from matplotlib import pyplot
import math
from joblib import Parallel, delayed
import multiprocessing
import gc
import random

In [2]:
current_dir = "./blue_whales/"

data_dir = current_dir + "data/"
labeled_data_dir = data_dir + 'labeled_data/'
audio_dir = data_dir + "raw_audio/"
output_spectrogram_dir = data_dir + "extracted_spectrogram/"

if not os.path.exists(output_spectrogram_dir):
    os.makedirs(output_spectrogram_dir)

In [ ]:
label_list = glob.glob(labeled_data_dir + '*.csv')
label_list

In [ ]:
ANT_calls = pd.read_csv(labeled_data_dir + 'ANT_calls.csv')
AUS_calls = pd.read_csv(labeled_data_dir + 'AUS_calls.csv')
MAD_calls = pd.read_csv(labeled_data_dir + 'MAD_calls.csv')
SRI_calls = pd.read_csv(labeled_data_dir + 'SRI_calls.csv')

all_calls = [ANT_calls, AUS_calls, MAD_calls, SRI_calls]
all_calls = pd.concat(all_calls).reset_index(drop=True)

print('Total number of labeled calls: ', len(all_calls))
all_calls.head()

In [5]:
print('Number of unique audio files with labels: ', len(all_calls.audio_filename.unique()))

Number of unique audio files with labels:  53


In [6]:
audio_list = glob.glob(audio_dir + '/*/*/*.wav')
print('Total number of unique audio files: ', len(audio_list))
print('Example: ', audio_list[0])

Total number of unique audio files:  55
Example:  E:/Blue_Whales/Data/Raw_Audio\ANT_BW\SWAMS2015\0635.wav


In [7]:
matching_audio_list = list(set([audio.split('\\')[-1] for audio in audio_list]) 
                           & 
                           set(all_calls.audio_filename.unique())
                          )
print('Number of matching audio files with labels', len(matching_audio_list))
print('Example: ', matching_audio_list[0])

Number of matching audio files with labels 53
Example:  0650.wav


In [8]:
spectrogram_seconds_duration = 240

def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    audio_length_second = int(len(sound_info) / frame_rate)
    wav.close()
    return sound_info, frame_rate, audio_length_second


In [9]:
def generate_spectrogram_positive(i):
    audio_filename = all_calls.loc[i, 'audio_filename']
    matching_audio_filename = [audio for audio in audio_list if audio_filename in audio][0]
    species = matching_audio_filename.split('raw_audio\\')[1][:3]
    detection_startSeconds = int(all_calls.loc[i, 'startSeconds'])
    detection_EndSeconds = int(all_calls.loc[i, 'EndSeconds'])
    spectrogram_startSeconds = max(0, detection_startSeconds - random.sample(range(0, 30), 1)[0])
    
    sound_info, frame_rate, audio_length_second = get_wav_info(matching_audio_filename)
    if spectrogram_startSeconds + spectrogram_seconds_duration <= audio_length_second:
        spectrogram_endSeconds = spectrogram_startSeconds + spectrogram_seconds_duration
    else:
        spectrogram_endSeconds = max(audio_length_second, detection_EndSeconds + random.sample(range(0, 30), 1)[0])
        spectrogram_startSeconds = spectrogram_endSeconds - spectrogram_seconds_duration
        
    pyplot.figure(num=None, figsize=(19, 12))
    pyplot.subplot(222)
    ax = pyplot.axes()
    ax.set_axis_off()
    pyplot.specgram(sound_info[frame_rate * spectrogram_startSeconds: frame_rate * spectrogram_endSeconds], 
                    Fs = frame_rate,
                    NFFT = 1024,
                    noverlap = 768)
    pyplot.savefig(output_spectrogram_dir + species + '_Positive_' + audio_filename.split('.')[0] + '_' + str(spectrogram_startSeconds) + '_' + str(spectrogram_endSeconds) + '.png', bbox_inches='tight', transparent=True, pad_inches=0.0)
    pyplot.close()
    
    gc.collect()
    

num_cores = multiprocessing.cpu_count()
spectrograms = Parallel(n_jobs=num_cores)(delayed(generate_spectrogram_positive)(i) for i in range(len(all_calls)))


In [10]:
def generate_spectrogram_negative(i):
    audio_filename = list(all_calls.audio_filename.unique())[i]
    matching_audio_filename = [audio for audio in audio_list if audio_filename in audio][0]
    species = matching_audio_filename.split('raw_audio\\')[1][:3]
    sound_info, frame_rate, audio_length_second = get_wav_info(matching_audio_filename)
    
    audio_filename_calls = all_calls.loc[all_calls.audio_filename == audio_filename]
    audio_filename_calls_startSeconds = audio_filename_calls.startSeconds.tolist()
    audio_filename_calls_EndSeconds = audio_filename_calls.EndSeconds.tolist()
    audio_filename_calls_startSeconds_EndSeconds = audio_filename_calls_startSeconds + audio_filename_calls_EndSeconds
    audio_filename_adjacent_calls_interval = []
    for j in range(len(audio_filename_calls_startSeconds) - 1):
        audio_filename_adjacent_calls_interval.append([audio_filename_calls_EndSeconds[j], audio_filename_calls_startSeconds[j+1]])
    
    for j in range(len(audio_filename_adjacent_calls_interval)):
        if audio_filename_adjacent_calls_interval[j][1] - audio_filename_adjacent_calls_interval[j][0] >= spectrogram_seconds_duration:
            audio_filename_EndSeconds_previous_call = math.ceil(audio_filename_adjacent_calls_interval[j][0])
            audio_filename_startSeconds_next_call = math.floor(audio_filename_adjacent_calls_interval[j][1])
            
            spectrogram_intervalSeconds = 60  ## generate spectrogram for every 60 seconds
            spectrogram_startSeconds = audio_filename_EndSeconds_previous_call + spectrogram_intervalSeconds
            while spectrogram_startSeconds + spectrogram_seconds_duration <= audio_filename_startSeconds_next_call:
                spectrogram_endSeconds = spectrogram_startSeconds + spectrogram_seconds_duration
                pyplot.figure(num=None, figsize=(19, 12))
                pyplot.subplot(222)
                ax = pyplot.axes()
                ax.set_axis_off()
                pyplot.specgram(sound_info[frame_rate * spectrogram_startSeconds: frame_rate * spectrogram_endSeconds], 
                                Fs = frame_rate,
                                NFFT = 1024,
                                noverlap = 768)
                pyplot.savefig(output_spectrogram_dir + species + '_Negative_' + audio_filename.split('.')[0] + '_' + str(spectrogram_startSeconds) + '_' + str(spectrogram_endSeconds) + '.png', bbox_inches='tight', transparent=True, pad_inches=0.0)
                pyplot.close()
                spectrogram_startSeconds += spectrogram_intervalSeconds

num_cores = multiprocessing.cpu_count()
spectrogram_no_calls = Parallel(n_jobs=num_cores)(delayed(generate_spectrogram_negative)(i) for i in range(len(all_calls.audio_filename.unique())))
 